In [4]:
import wget
import wsgiref
import requests
import urllib.parse as parse
import lxml.html
import urllib
import string

header = {'user-agent': 'Mozilla'}

In [4]:
fname = "https://academic.oup.com/gji/article-pdf/215/2/942/25504851/ggy316.pdf"

wget.download(fname, out='gji.pdf')

'gji.pdf'

In [47]:
r = requests.post('http://httpbin.org/post', data = {'key':'value'})



In [48]:
wget.

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "key": "value"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Connection": "close", \n    "Content-Length": "9", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.18.4"\n  }, \n  "json": null, \n  "origin": "134.245.134.211", \n  "url": "http://httpbin.org/post"\n}\n'

In [136]:
dest = "https://academic.oup.com/gji/downloadcitation/5061121"
r = requests.get(dest, headers={'user-agent': 'Mozilla'}, params={'format':'bibtex'}) #, allow_redirects=True)

In [137]:
r.content.splitlines()

[b'@article{doi:10.1093/gji/ggy316,',
 b'author = {Fan, Wenyuan and de\xc2\xa0Groot-Hedlin, Catherine D and Hedlin, Michael A H and Ma, Zhitu},',
 b'title = {Using surface waves recorded by a large mesh of three-element arrays to detect and locate disparate seismic sources},',
 b'journal = {Geophysical Journal International},',
 b'volume = {215},',
 b'number = {2},',
 b'pages = {942-958},',
 b'year = {2018},',
 b'doi = {10.1093/gji/ggy316},',
 b'URL = {http://dx.doi.org/10.1093/gji/ggy316},',
 b'eprint = {/oup/backfile/content_public/journal/gji/215/2/10.1093_gji_ggy316/2/ggy316.pdf}',
 b'}']

In [114]:
doiurl = 'https://doi.org/10.1093/gji/ggy316'
dr = requests.get(doiurl, headers={'user-agent': 'Mozilla'})

In [118]:
article_url = 'https://academic.oup.com/gji/article/215/2/942/5061121'

In [143]:
p = parse.urlsplit(article_url)
print(p)
journal = p.path.split('/')[1]
article_id = p.path.split('/')[-1]
cit_path = '/'+journal+'/downloadcitation/'+article_id
cit_url = parse.urlunsplit((p.scheme, p.netloc, cit_path, p.query, p.fragment))

pdf_path = p.path.replace('article', 'article-pdf')
pdf_url = parse.urlunsplit((p.scheme, p.netloc, pdf_path, p.query, p.fragment))
print(cit_url)
print(pdf_url)

SplitResult(scheme='https', netloc='academic.oup.com', path='/gji/article/215/2/942/5061121', query='', fragment='')
https://academic.oup.com/gji/downloadcitation/5061121
https://academic.oup.com/gji/article-pdf/215/2/942/5061121


# Working flow for GJI

In [385]:
article_url = 'https://academic.oup.com/gji/article/215/2/1037/5061122'


url_elems = parse.urlsplit(article_url)
website = requests.get(article_url, headers=header)

dom =  lxml.html.fromstring(website.text)
dom.make_links_absolute(parse.urlunsplit((url_elems.scheme, url_elems.netloc,'', '', '')))

In [386]:
links = dom.xpath('//a/@href')

foundpdf = False
foundbibtex = False

while not (foundpdf and foundbibtex):
    link = links.pop()
    if 'article-pdf' in link:
        link2pdf = link
        foundpdf = True
    elif 'format=bibtex' in link:
        link2bibtex = link
        foundbibtex = True
print (link2bibtex)
print(link2pdf)
#for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
#    print (link)

https://academic.oup.com/gji/downloadcitation/5061122?format=bibtex
https://academic.oup.com/gji/article-pdf/215/2/1037/25602679/ggy310.pdf


In [388]:
cit_url = parse.splitquery(link2bibtex)[0]
bibitem = requests.get(cit_url, headers={'user-agent': 'Mozilla'}, params={'format':'bibtex'})
bibitem.content.splitlines()

[b'@article{doi:10.1093/gji/ggy310,',
 b'author = {Gao, Chao and Leki\xc4\x87, Vedran},',
 b'title = {Consequences of parametrization choices in surface wave inversion: insights from transdimensional Bayesian methods},',
 b'journal = {Geophysical Journal International},',
 b'volume = {215},',
 b'number = {2},',
 b'pages = {1037-1063},',
 b'year = {2018},',
 b'doi = {10.1093/gji/ggy310},',
 b'URL = {http://dx.doi.org/10.1093/gji/ggy310},',
 b'eprint = {/oup/backfile/content_public/journal/gji/215/2/10.1093_gji_ggy310/1/ggy310.pdf}',
 b'}']

In [405]:
title = dom.find_class("wi-article-title article-title-main")[0].text.strip()  # Get Title

translator = str.maketrans('', '', string.punctuation)  # Remove punctuation characters
title = title.translate(translator)
title = title.replace(' ', '_')[:100]  # Use "_" instead of ehitespace for pdf-filename

author = dom.find_class("linked-name")[0].text.strip() # Get first author, remove whitespace
author = author.split(' ')[-1]  # Keep only last name

pdate = dom.find_class('citation-date')[0].text.strip()  # Get puplication date
pday, pmonth, pyear = pdate.split(' ')  

pretty_pdfname = '{}-{}-{}.pdf'.format(author, pyear, title)

print(author, title, pyear)

print(pretty_pdfname)

string.punctuation
#pdfclass = dom.find_class("article-pdfLink")[0]
#article_id = pdfclass.get('data-article-id')
#pdflink = pdfclass.get('href')
#print(pdflink)




Gao Consequences_of_parametrization_choices_in_surface_wave_inversion_insights_from_transdimensional_Bay 2018
Gao-2018-Consequences_of_parametrization_choices_in_surface_wave_inversion_insights_from_transdimensional_Bay.pdf


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [409]:
## Could also be done with wget in a one-liner but maybe wget not flexible enough?
pdf = requests.get(link2pdf, headers={'user-agent': 'Mozilla'})
with open(pretty_pdfname, 'wb') as f:
    f.write(pdf.content)

# AGU

In [517]:
article_url = 'https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2018GL078991'
url_elems = parse.urlsplit(article_url)
website = requests.get(article_url, headers=header)

dom =  lxml.html.fromstring(website.text)
dom.make_links_absolute(parse.urlunsplit((url_elems.scheme, url_elems.netloc,'', '', '')))

In [518]:
links = dom.xpath('//a/@href')

foundpdf = False
foundbibtex = True

while not (foundpdf and foundbibtex):
    link = links.pop()
    if '/pdf/' in link:
        link2pdf = link
        foundpdf = True
    elif '/action/showCitFormats' in link:  # Unnecessary
        link2bibtex = link
        foundbibtex = True
print(link2bibtex)
print(link2pdf)

doi = link2pdf.split('/')[-2:]
doi = '/'.join(doi)
print(doi)

https://agupubs.onlinelibrary.wiley.com/action/showCitFormats?doi=10.1029%2F2018GL078991
https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2018GL078991
10.1029/2018GL078991


In [421]:
cit_query = 'https://agupubs.onlinelibrary.wiley.com/action/showCitFormats'
r = requests.get(cit_query, headers=header, params={'doi':'10.1029%2F2018GL078991'})

In [467]:
bibitem = requests.get('https://agupubs.onlinelibrary.wiley.com/action/downloadCitation', headers={'user-agent': 'Mozilla'}, params={'direct':'direct', 'doi':doi, 'downloadFileName':'pericles_19448007AxA', 
                                                                               'format':'bibtex', 'include':'abs', 'submit':'Download'})

bibitem.text.strip().splitlines()

In [475]:
title = dom.find_class('citation__title')[0].text.strip()
title

'How Fast Can We Reliably Estimate the Magnitude of Subduction Earthquakes?'

In [507]:
author = dom.xpath('/html/body/div[1]/div/div[2]/div/div/div[2]/div/div/div/div/article/div/div[1]/div[2]/div/div[4]/div/div/div[1]/a/span')[0].text

In [516]:
date = dom.find_class('epub-date')[0].text
print(pdate)

10 September 2018


In [515]:
date[0].text

'10 September 2018'

# Elsevier ... sucks.

In [9]:
def get_page_content(article_url, headers=header):
    url_elems = parse.urlsplit(article_url)
    website = requests.get(article_url, headers=header)

    dom =  lxml.html.fromstring(website.text)
    dom.make_links_absolute(parse.urlunsplit((url_elems.scheme, url_elems.netloc,'', '', '')))
    return dom


def get_hrefs(dom, pdfmark, bibtexmark):
    links = dom.xpath('//a/@href')

    foundpdf = False
    foundbibtex = True

    while not (foundpdf and foundbibtex):
        link = links.pop()
        if pdfmark in link:
            link2pdf = link
            foundpdf = True
        elif bibtexmark in link:  # Unnecessary
            link2bibtex = link
            foundbibtex = True
    print(link2bibtex)
    print(link2pdf)

    doi = link2pdf.split('/')[-2:]
    doi = '/'.join(doi)
    print(doi)


In [17]:
article_url = 'https://www.sciencedirect.com/science/article/pii/S0377027312001448?via%3Dihub'
dom = get_page_content(article_url)

In [34]:
links = dom.xpath('//a/@href')
links

['https://www.sciencedirect.com#app',
 'https://www.sciencedirect.com',
 'https://www.sciencedirect.com',
 'https://www.sciencedirect.com/',
 'https://www.sciencedirect.com/browse/journals-and-books',
 'https://www.sciencedirect.com/user/register?returnURL=%2Fscience%2Farticle%2Fpii%2FS0377027312001448',
 'https://www.sciencedirect.com/user/login?returnURL=%2Fscience%2Farticle%2Fpii%2FS0377027312001448',
 'https://service.elsevier.com/app/home/supporthub/sciencedirect/',
 'https://www.sciencedirect.com',
 'https://www.sciencedirect.com',
 'https://www.sciencedirect.com/browse/journals-and-books',
 'https://www.sciencedirect.com/user/register?returnURL=%2Fscience%2Farticle%2Fpii%2FS0377027312001448',
 'https://www.sciencedirect.com/user/login?returnURL=%2Fscience%2Farticle%2Fpii%2FS0377027312001448',
 'https://service.elsevier.com/app/home/supporthub/sciencedirect/',
 'https://www.sciencedirect.com/search/advanced',
 'https://www.sciencedirect.com/science/journal/03770273',
 'https://ww

In [33]:
p = dom.find_class('PdfDropDownMenu')
p

[]

In [35]:
aurl = 'https://www.sciencedirect.com/science/article/pii/S0377027312001448/pdfft' # ?md5=6db53957edf22d0f5a49fa0250e6be31&pid=1-s2.0-S0377027312001448-main.pdf
site = get_page_content(aurl)

In [42]:
site.xpath('//a/@href')

['https://ac.els-cdn.com/S0377027312001448/1-s2.0-S0377027312001448-main.pdf?_tid=38960f33-9c2b-4cca-8f54-084e9fb09643&acdnat=1537953106_65c1e006d3587bb3b35a26d7049f9c1c']

In [113]:
base = 'https://www.sciencedirect.com'
pdfref = base+"/science/article/pii/S0377027312001448/pdfft?md5=6db53957edf22d0f5a49fa0250e6be31&amp;pid=1-s2.0-S0377027312001448-main.pdf"
url, query = parse.splitquery(pdfref)

params = {}
for param in query.split('&'):
    params[param.split('=')[0]] = param.split('=')[1]

r = requests.get(url, headers=header, params=params, allow_redirects=True, timeout=10 )

In [116]:
r.content.splitlines()

[b'<!DOCTYPE html>',
 b'<html class="no-js no-svg" lang="en-US">',
 b'    <head>',
 b'        <title>ScienceDirect.com | Science, health and medical journals, full text articles and books.</title>',
 b'',
 b'        <link rel="dns-prefetch" href="//www.sciencedirect.com"/>',
 b'        <link rel="canonical" href="http://www.sciencedirect.com/"/>',
 b'        <link rel="shortcut icon" href="https://sdfestaticassets-us-east-1.sciencedirectassets.com/shared-assets/15/images/favSD.ico" type="image/x-icon"/>',
 b'        <link rel="icon" href="https://sdfestaticassets-us-east-1.sciencedirectassets.com/shared-assets/15/images/favSD.ico" type="image/x-icon">',
 b'        <script id= "adobe-dtm" src="https://assets.adobedtm.com/376c5346e33126fdb6b2dbac81e307cbacfd7935/satelliteLib-b7cfe8df39a4e5eec5536bba80e13f4b6fa0dd7c.js" type="text/javascript"></script>',
 b'',
 b'    </head>',
 b'',
 b'    <body>',
 b'        <noscript>',
 b'            <div id="redirect-message">',
 b'                <p>

In [90]:
r = requests.get(aurl, headers=header, allow_redirects=True, timeout=10)

In [89]:
<a class="anchor" href="/science/article/pii/S0377027312001448/pdfft?md5=6db53957edf22d0f5a49fa0250e6be31&amp;pid=1-s2.0-S0377027312001448-main.pdf" target="_blank" rel="nofollow" aria-expanded="true" aria-live="polite" aria-label="Download single PDF. Opens in a new window."><svg focusable="false" viewBox="0 0 32 32" width="24" height="24" class="icon icon-pdf-multicolor"><path d="M7 .362h17.875l6.763 6.1V31.64H6.948V16z" stroke="#000" stroke-width=".703" fill="#fff"></path><path d="M.167 2.592H22.39V9.72H.166z" stroke="#aaa" stroke-width=".315" fill="#da0000"></path><path fill="#fff9f9" d="M5.97 3.638h1.62c1.053 0 1.483.677 1.488 1.564.008.96-.6 1.564-1.492 1.564h-.644v1.66h-.977V3.64m.977.897v1.34h.542c.27 0 .596-.068.596-.673-.002-.6-.32-.667-.596-.667h-.542m3.8.036v2.92h.35c.933 0 1.223-.448 1.228-1.462.008-1.06-.316-1.45-1.23-1.45h-.347m-.977-.94h1.03c1.68 0 2.523.586 2.534 2.39.01 1.688-.607 2.4-2.534 2.4h-1.03V3.64m4.305 0h2.63v.934h-1.657v.894H16.6V6.4h-1.56v2.026h-.97V3.638"></path><path d="M19.462 13.46c.348 4.274-6.59 16.72-8.508 15.792-1.82-.85 1.53-3.317 2.92-4.366-2.864.894-5.394 3.252-3.837 3.93 2.113.895 7.048-9.25 9.41-15.394zM14.32 24.874c4.767-1.526 14.735-2.974 15.152-1.407.824-3.157-13.72-.37-15.153 1.407zm5.28-5.043c2.31 3.237 9.816 7.498 9.788 3.82-.306 2.046-6.66-1.097-8.925-4.164-4.087-5.534-2.39-8.772-1.682-8.732.917.047 1.074 1.307.67 2.442-.173-1.406-.58-2.44-1.224-2.415-1.835.067-1.905 4.46 1.37 9.065z" fill="#f91d0a"></path></svg><span class="anchor-text">Article</span></a>

IndexError: list index out of range

In [73]:
<a class="anchor" href="/science/article/pii/S0377027312001448/pdfft?md5=6db53957edf22d0f5a49fa0250e6be31&amp;pid=1-s2.0-S0377027312001448-main.pdf" target="_blank" rel="nofollow" aria-expanded="true" aria-live="polite" aria-label="Download single PDF. Opens in a new window."><svg focusable="false" viewBox="0 0 32 32" width="24" height="24" class="icon icon-pdf-multicolor"><path d="M7 .362h17.875l6.763 6.1V31.64H6.948V16z" stroke="#000" stroke-width=".703" fill="#fff"></path><path d="M.167 2.592H22.39V9.72H.166z" stroke="#aaa" stroke-width=".315" fill="#da0000"></path><path fill="#fff9f9" d="M5.97 3.638h1.62c1.053 0 1.483.677 1.488 1.564.008.96-.6 1.564-1.492 1.564h-.644v1.66h-.977V3.64m.977.897v1.34h.542c.27 0 .596-.068.596-.673-.002-.6-.32-.667-.596-.667h-.542m3.8.036v2.92h.35c.933 0 1.223-.448 1.228-1.462.008-1.06-.316-1.45-1.23-1.45h-.347m-.977-.94h1.03c1.68 0 2.523.586 2.534 2.39.01 1.688-.607 2.4-2.534 2.4h-1.03V3.64m4.305 0h2.63v.934h-1.657v.894H16.6V6.4h-1.56v2.026h-.97V3.638"></path><path d="M19.462 13.46c.348 4.274-6.59 16.72-8.508 15.792-1.82-.85 1.53-3.317 2.92-4.366-2.864.894-5.394 3.252-3.837 3.93 2.113.895 7.048-9.25 9.41-15.394zM14.32 24.874c4.767-1.526 14.735-2.974 15.152-1.407.824-3.157-13.72-.37-15.153 1.407zm5.28-5.043c2.31 3.237 9.816 7.498 9.788 3.82-.306 2.046-6.66-1.097-8.925-4.164-4.087-5.534-2.39-8.772-1.682-8.732.917.047 1.074 1.307.67 2.442-.173-1.406-.58-2.44-1.224-2.415-1.835.067-1.905 4.46 1.37 9.065z" fill="#f91d0a"></path></svg><span class="anchor-text">Article</span></a>

[b'',
 b'      <!doctype html>',
 b"      <html lang='en-us'>",
 b'          <head>',
 b'              <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">',
 b'              <meta name="SDTech" content="Proudly brought to you by the SD Technology team in London, Dayton, and Amsterdam">',
 b"              <meta charset='utf-8'>",
 b'              <meta name="robots" content="NOINDEX,FOLLOW,NOARCHIVE,NOODP,NOYDIR">',
 b'              <title>Internal Server Error</title>',
 b'              <link rel="icon" href="https://sdfestaticassets-us-east-1.sciencedirectassets.com/shared-assets/15/images/favSD.ico" type="image/x-icon" />',
 b"              <link rel='stylesheet' href=https://sdfestaticassets-us-east-1.sciencedirectassets.com/prod/2efdf6e3b3bfafb2a279f3a295538060ed05b2c8/style.css>",
 b'              ',
 b'              <script src="//cdn.optimizely.com/js/204774041.js"></script>',
 b'              <!-- begin mPulse embed code -->',
 b'      <script>